# Parameters, data load and initial conditioning.

First, we add all the necessary packages.

In [1]:
using Pkg
Pkg.activate(".")

using PythonCall,CondaPkg
JEft=pyimport("justetf_scraping")

using CairoMakie
using Serialization, CSV, DataFrames, Dates
using Statistics, CovarianceEstimation, Random
using LinearAlgebra 
using NLopt

include("src/src.jl")

  Activating project at `~/Codes/portfolio`


InitError: InitError: expected package `IJulia [7073ff75]` to exist in the manifest (use `resolve` to populate the manifest)
during initialization of module C

Parameters:
- `shift`: a period of time in days over which all the returns will be evaluated.
- `risk_free_return`: the return over `shift` period of time considered risk free, e.g. bank deposit.
- `half_life`: to compute mean return over `shift` period we use weighted means. The data at the day `today-half_life` will have weight two times smaller than the data from `today`. Units - number of `shift`s. 
- `correlation_threshold`: at some moment we will filter our tickers so that no tickers with correlation above the given threshold are left. For faster results use lower threshold!
- `update_charts_q`: if `true`, the fresh charts from justETF will be downloaded (this will take around 30 minutes). If `false`, the charts will be loaded from the data folder. If there is no charts there, charts will be loaded from justETF.
- `data_starting_date`: ETS that don't have data earlier than this will be thrown away.
- `available_tickers_file`: location of the file that lists the tickers we have an access to. If `nothing`, all tickers from the justETF website will be considered available. The file should be a table with "isin" column.

In [ ]:
shift=365;
risk_free_return=0.037;
half_life=4;
correlation_threshold=0.7;
update_charts_q=true;
data_starting_date=Date(2019);
available_tickers_file=nothing;

We create two tables with data:
 - `DO` stands for "Data Overview". There, each row corresponds to a particular ticker. Contains some general information.
 - `DC` stands for "Data Charts". There, the first column contains all the dates and the rest of the columns corresponds to values of tickers.

In [ ]:

tmp=JEft.load_overview(strategy="epg-longOnly")
tmp.to_csv("tmp.csv")
DO=CSV.read("tmp.csv", DataFrame)
rm("tmp.csv");

if update_charts_q
    DC=up_to_date_charts(DO)
else
    if isfile("data/charts.csv")
        DC=CSV.read("data/charts.csv", DataFrame)
    else
        DC=up_to_date_charts(DO)
    end
end

tickers_consistent_with_DC=DO.isin .|> x-> x in names(DC)
if !all(tickers_consistent_with_DC)
    println("Attention, new tickets in the overview table. The database is outdated. Ignore, or update the database.")
end

Now, we create the conditioned tables `DO_acc` and `DC_acc`. In these tables we have only accumulating ETFs with the fund size of at least 50 millions euro. Furthermore, we throw away ETFs which don't have data at least up to `data_starting_date` unless we can compensate it using the data from distributing ETF version.

In [ ]:
conditions=(DO.dividends.=="Accumulating") .& (DO.size .|> x -> ismissing(x) ? false : x>=50 ) .& tickers_consistent_with_DC

DO_acc=DO[conditions,:]
DC_acc=DC[:,["date",DO_acc.isin...]]


tickers_to_remove=Vector{Int}()
for i=2:length(names(DC_acc))
    tk=names(DC_acc)[i]
    tk_vals=DC_acc[!,tk]

    tk_first_day_index=findfirst(x->!(ismissing(x)),tk_vals)

    if DC_acc[tk_first_day_index,"date"]>data_starting_date
        tk_dist_name=dist_name(DO[DO.isin.==tk,:name]...)
        tk_dist=begin tmp = DO[DO.name.==tk_dist_name,:isin]; length(tmp)>0 ? tmp[1] : nothing end
        
        if !(isnothing(tk_dist))
            tk_dist_vals=DC[!,tk_dist]
            tk_dist_first_day_index=findfirst(x->!(ismissing(x)),tk_dist_vals)
            if DC[tk_dist_first_day_index,"date"] <= data_starting_date
                DC_acc[!,tk].=DC[!,tk_dist]
                println("Data in $tk was replaced with data in $(tk_dist)")
            end
            println("$tk does not have enough data and its distributing version does not have it either. $tk will be removed from the database.")
            push!(tickers_to_remove,i-1)
        else
            println("$tk does not have enough data and there is no distributing version of it to compensate. $tk will be removed from the database.")
            push!(tickers_to_remove,i-1)
        end
    else
        println("$tk has enough data")
    end
end

println("Removing tickers with not enough data")
DO_acc=DO_acc[Not(tickers_to_remove),:]
DC_acc=DC_acc[:,["date",DO_acc.isin...]];


if !isnothing(available_tickers_file)
    println("Removing unavailable tickers")

    available_tickers=CSV.read(available_tickers_file, DataFrame).isin

    availability_filter=DO_acc.isin .|> x-> x ∈ available_tickers

    DO_acc=DO_acc[availability_filter,:]
    DC_acc=DC_acc[:,["date",DO_acc.isin...]];
end;

Let us take a random ticker and plot its behavior to compare with the website.

In [ ]:
tk=rand(names(DC_acc)[2:end])
println("ticker: ", tk)
fig=Figure(;size=(1200,300))
ax=Axis(fig[1,1], title=tk)
lines!(ax,DC_acc[!,:date],DC_acc[!,tk].-100; color=:black)
fig

# Decorrelation

Now, we will reduce the number of tickers further by dividing them into highly correlated groups and choosing only one ticker from each group (the one with the highest Sharpe ratio). For this, we will need compute the expectation values `R` and covariance matrix `RR` of the annual returns. We also compute the correlation matrix `cor`, which is a normalized version of the covariance matrix. We will use `cor` to distinguish the correlated assets and `R`, `RR` to compute the Sharpe ratio. 

We compute the covariance matrix by using so called shrinkage estimators. I don't know much about these and use the method here more or less blindly. The idea is that if we don't have a lot of data point (comparing with the number of tickers) then the standard method can be unstable. Shrinkage estimators are supposed to make the result a bit more reliable.

In [ ]:
R,RR,cor=get_R_RR_cor(DC_acc, shift, half_life);
cor

We have now correlation matrix, let us look at how tickers with correlations above certain threshold behave. The following cell will chose a random ticker, take `nsample` tickers whose correlation with it lies inside the `correlation_interval` and plot their behavior. The red line is the chosen ticker and the gray lines is the correlated tickers. To make the comparison simpler, all the lines are shifted so that the last point is at zero.

In [ ]:
correlation_interval=[0.9,1.0]
nsample=100

i=rand(1:size(DO_acc,1))
tk=names(DC_acc)[i+1]

println("chosen ticker: $tk")

tk_cor=cor[:,i]

if count(tk_cor .|> (x-> x>correlation_interval[1] && x<correlation_interval[2]))>1

    correlated_tickers=names(DC_acc)[2:end][tk_cor .|> (x-> x>correlation_interval[1] && x<correlation_interval[2])]
    correlation_values=tk_cor[tk_cor .|> (x-> x>correlation_interval[1] && x<correlation_interval[2])]

    correlation_color=(correlation_values.-findmin(correlation_values)[1])./(findmax(correlation_values)[1]-findmin(correlation_values)[1])
    println("amount of tickers in the correlation interval: $(length(correlated_tickers))")

    fig=Figure(;size=(1800,450))
    ax=Axis(fig[1,1], title="price")

    for i in 1:min(nsample,length(correlated_tickers))
        tkc=correlated_tickers[i]    
        sh=collect(skipmissing(DC_acc[:,tkc]))[end]
        lines!(ax,DC_acc[!,"date"], DC_acc[!,tkc].-sh,color=(:black, (correlation_color[i])))
    end

    sh=collect(skipmissing(DC_acc[:,tk]))[end]
    lines!(ax,DC_acc[!,"date"], DC_acc[!,tk].-sh, color=:red)
    
    ax=Axis(fig[1,2], title="relative return over $shift days")


    for i in 1:min(nsample,length(correlated_tickers))
        tkc=correlated_tickers[i]    
        lines!(ax, DC_acc[(1+shift):end,"date"] ,percent_return(DC_acc[:,tkc],shift),color=(:black, (correlation_color[i])))
    end

    lines!(ax,DC_acc[(1+shift):end,"date"], percent_return(DC_acc[!,tk],shift), color=:red)

    color_range = (minimum(correlation_values),maximum(correlation_values))
    cmap = :grays

    cbar = Colorbar(fig[1,3], colormap=Reverse(cmap), limits=color_range,
    flipaxis=true, label="Correlation")


    fig 
else
    println("Ticker is not correlated with other tickers in the given correlation interval, please try again.")
end


The cell below runs the correlation reducing procedure resulting in the data collection `DC_acc_cr` such that non of two tickers in `DC_acc_cr` are correlated above `correlation_theshold`. After the procedure is done, we also define the data collection `DO_acc_cr` with general information about tickers from `DC_acc_cr`. 

In [ ]:
DC_acc_cr=copy(DC_acc)
R,RR,cor=get_R_RR_cor(DC_acc_cr, shift, half_life);

while length(cor[cor.>correlation_threshold])>size(cor,1)
    R,RR,cor=get_R_RR_cor(DC_acc_cr, shift, half_life)
    println("number of tickers: ", size(cor,1))


    cnt=1
    ct_filter=(cor[:,cnt].>correlation_threshold)
    while cnt<=length(R)
        ct_filter=(cor[:,cnt].>correlation_threshold)
        if count(ct_filter)>1
            break
        end
        cnt+=1
    end

    if cnt>length(R)
        break
    end

    correlated_tickers_ind=findall(x->x==true, ct_filter)

    sharpe_ratios=[]
    for tk ∈ correlated_tickers_ind
        push!(sharpe_ratios,(R[tk]-risk_free_return)/(sqrt(RR[tk,tk])))
    end
    best_ticker_ind=correlated_tickers_ind[findmax(sharpe_ratios)[2]]
    
    filter!(x->x!=best_ticker_ind,correlated_tickers_ind)

    tickers_to_remove=names(DC_acc_cr)[2:end][correlated_tickers_ind]

    DC_acc_cr=DC_acc_cr[!,Not(tickers_to_remove)]
end

DO_acc_cr=DO_acc[DO_acc.isin .|> (x->x ∈ names(DC_acc_cr)), :]

# Optimization

Before going to the optimization, let us define some parameters:
 - `USD_bound`: gives a constrain on the total amount of the USD based tickers
 - `entropy_factor`: we add entropy times `entropy_factor` into the objective function to improve diversification.
 - `lowest_share`: if not zero, the code will run optimization twice. In the second time, only tickers with share (in the previous solution) larger than `lowest_share` will be used.
 - `xtol_rel_1`: In the first optimization round, if the relative change of the weight vector after a step is less than `xtol_rel_1` the search stops
 - `xtol_rel_2`: In the second optimization round, if the relative change of the weight vector after a step is less than `xtol_rel_2` the search stops

In [ ]:
USD_bound=0.5
entropy_factor=0.2;
lowest_share=0.01;
xtol_rel_1=1e-4;
xtol_rel_2=1e-6;

Now, let us proceed to the optimization. It is still semi-global. ISRES algorithm does not guaranties to find global optimum, but it tries to avoid local traps.

In [ ]:
R,RR,cor=get_R_RR_cor(DC_acc_cr, shift, half_life)
USD_filter=DO_acc_cr.currency.=="USD"


function entropy(w)
    -sum((w).*log.(w))
end
function sharpe_ratio(w) 
    return (dot(R,w)-risk_free_return)/sqrt(dot(w,RR*w))
end
function objective(w, grad)
    if length(grad) > 0
        wCw_root=(dot(w,RR*w))^1/2
        grad.=R/wCw_root-(dot(R,w)-risk_free_return)*(RR*w)/(wCw_root)^3-entropy_factor*(log.(w).+1)
    end
    return sharpe_ratio(w)+entropy_factor*entropy(w)
end
function sum_constrain(w, grad)
    if length(grad) > 0
        grad.=1
    end
    return sum(w)-1
end
function USD_constrain(w,grad)
    if length(grad) > 0
        grad.=USD_filter
    end
    return sum(w[USD_filter])-USD_bound
end


opt = NLopt.Opt(:GN_ISRES, length(R))
NLopt.lower_bounds!(opt, zeros(length(R)))
NLopt.upper_bounds!(opt, ones(length(R)))
equality_constraint!(opt, sum_constrain, 1e-8)
inequality_constraint!(opt, USD_constrain, 1e-8)

NLopt.max_objective!(opt, objective)

NLopt.xtol_rel!(opt, xtol_rel_1)

w_opt=rand(length(R))
w_opt/=sum(w_opt)

max_f, w_opt, ret = NLopt.optimize!(opt, w_opt)
num_evals = NLopt.numevals(opt)

ret_risk_r=dot(R,w_opt)/sqrt(dot(w_opt,RR*w_opt))

println(
    """
    First round of optimization (ISRES):
    solution status           : $ret
    sum of weights            : $(sum(w_opt))
    USD share                 : $(sum(w_opt[USD_filter]))
    # function evaluation     : $num_evals
    -------------------------------------------------
    objective value           : $max_f
    entropy                   : $(entropy(w_opt))
    mean return               : $(dot(R,w_opt))
    mean risk                 : $(sqrt(dot(w_opt,RR*w_opt)))
    return/risk ratio         : $(ret_risk_r)
    """
)

low_share_filter=(w_opt.>=lowest_share)
if lowest_share!=0
    println("The optimization problem will be restricted to $(count(low_share_filter)) tickers with share more than $lowest_share")
    R=R[low_share_filter]
    RR=RR[low_share_filter,low_share_filter]
    w_opt=w_opt[low_share_filter]
    w_opt./=sum(w_opt)
    USD_filter=USD_filter[low_share_filter]

    DO_acc_cr=DO_acc_cr[low_share_filter,:]
    DC_acc_cr=DC_acc_cr[:,["date",DO_acc_cr.isin...]];
end

opt = NLopt.Opt(:GN_ISRES, length(R))

NLopt.lower_bounds!(opt, ones(length(R))*lowest_share)
NLopt.upper_bounds!(opt, ones(length(R)))
equality_constraint!(opt, sum_constrain, 1e-8)
inequality_constraint!(opt, USD_constrain, 1e-8)

NLopt.max_objective!(opt, objective)


NLopt.xtol_rel!(opt, xtol_rel_2)

max_f, w_opt, ret = NLopt.optimize!(opt, w_opt)
num_evals = NLopt.numevals(opt)

ret_risk_r=dot(R,w_opt)/sqrt(dot(w_opt,RR*w_opt))


println(
    """
    Second round of optimization (ISRES):
    solution status           : $ret
    sum of weights            : $(sum(w_opt))
    USD share                 : $(sum(w_opt[USD_filter]))
    # function evaluation     : $num_evals
    -------------------------------------------------
    objective value           : $max_f
    entropy                   : $(entropy(w_opt))
    mean return               : $(dot(R,w_opt))
    mean risk                 : $(sqrt(dot(w_opt,RR*w_opt)))
    return/risk ratio         : $(ret_risk_r)
    """
)

In [ ]:
p=sortperm(w_opt; rev=true)
names_ordered=DO_acc_cr.name[p]
isin_ordered=DO_acc_cr.isin[p]
w_opt_ord=w_opt[p]
for i=1:length(w_opt_ord)
    println(names_ordered[i]," : ", isin_ordered[i], " | ", round(w_opt_ord[i]; digits=3))
end

Let us now look at the historical performance of this portfolio (with weights rounded to 3 digits).

In [ ]:
w_opt_round=round.(w_opt; digits=3);
share_color=(w_opt_round.-findmin(w_opt_round)[1])./(findmax(w_opt_round)[1]-findmin(w_opt_round)[1])

tickers_in_portfolio=DO_acc_cr.isin
DC_acc_cr_nrm=dropmissing(DC_acc_cr[:, ["date",tickers_in_portfolio...]])
mapcols!(x->x.-x[1].+100,DC_acc_cr_nrm; cols=Not("date"))

portfolio_value=sum(Matrix(reshape(w_opt_round,1,length(w_opt_round)).*DC_acc_cr_nrm[!,Not("date")]); dims=2)[:]

fig=Figure(;size=(1200,300))
ax=Axis(fig[1,1], title="Portfolio")

# Plot constituents first
constituent_line = nothing
for tk in names(DC_acc_cr_nrm)[2:end][w_opt_round.>1e-10]
    tk_ind=findfirst(x->x==tk,names(DC_acc_cr_nrm)[2:end])
    constituent_line = lines!(ax,DC_acc_cr_nrm[!,"date"], DC_acc_cr_nrm[!,tk].-100; color=(:black,share_color[tk_ind]))
end

# Plot portfolio
portfolio_line = lines!(ax,DC_acc_cr_nrm[!,"date"], portfolio_value.-100; color=:red)

# Add legend with explicit colors
axislegend(ax, [LineElement(linewidth=2, color=:red), LineElement(linewidth=2, color=:black)], ["portfolio", "constituents"], position=:lt)

# Add colorbar for shares
cbar = Colorbar(fig[1, 2], limits=(minimum(w_opt_round), maximum(w_opt_round)),
                colormap=Reverse(:grays), label="Share")

fig

Let us safe the result of the optimization together with the data used for it.

In [ ]:
serialize("portfolio", Dict("overview_table"=>DO_acc_cr, "optimal_portfolio"=>w_opt, "USD_bound"=>USD_bound, "entropy_factor"=>entropy_factor, "lowest_share"=>lowest_share))

# Improvement ideas

## performance

1. Understand better the covariance matrix computation as now we are using it blindly.
2. For now we are using semi-global optimization technique. It does have a way to avoid local optima, however, there is no guarantee that it will not stuck there for sure. Can we do better?    
3. It would be nice to compare the perfromance of the optimized portfolio with its perturbations to understand how much of the deviation is tolerable and when one must do rebalancing.
   
## cosmetics

1. It would be nice to see the first and the last dates on the historical plot. 